In [1]:
# Libraries
from azure.storage.blob import BlobServiceClient
from datetime import datetime
import json
import os
from pathlib import Path
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import shutil
import time
import urllib.request, urllib.error, urllib.parse
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

In [2]:
# Set up Chrome options
chrome_options = Options()
# chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument("--headless")
# chrome_options.add_argument("--no-sandbox")

In [3]:
html_info = []
html_folder = "html_file"
csv_folder = "generated_csv"
acct_url = "https://tokyoolymphicdata.blob.core.windows.net"
container_name = 'web-scraping'
container_sas_key = "sp=racwdlmeo&st=2024-09-27T12:09:32Z&se=2026-09-27T20:09:32Z&spr=https&sv=2022-11-02&sr=c&sig=jhSYaHX2ARyEupmEqpQZKLKbuX4I6EuLpHg1ABdgiZg%3D"
target_dir_name = "active_dir"
blob_service_client = BlobServiceClient(account_url= acct_url, credential=container_sas_key)
container_client = blob_service_client.get_container_client(container_name)

In [4]:
# Download HTML
# Iterate Filters
def filter_wise_func_call():
    filters_list = ["Earthquakes","Floods","Cyclones","Volcanoes","Droughts","Wildfires"]
    filters_tags_list = ["inputChEq","inputChFl","inputChTc","inputChVo","inputChDr","inputChFf"]
    for filter,filter_tag in zip(filters_list,filters_tags_list):
        links_iterator(filter,filter_tag)

In [5]:

# Selenium HTML file navigation
def links_iterator(filter,filter_tag):
    print(filter,filter_tag,"checking")
    start_date = "2020-01-01"
    driver = webdriver.Chrome(chrome_options)
    # Open the GDACS website
    driver.get("https://www.gdacs.org/")
    # Wait until the page loads and the element is present
    wait = WebDriverWait(driver, 5)
    alert_tab = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@class="navbar-nav"]/a[2]')))
    alert_tab.click()
    # Click the Cookies banner
    btn = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="cookie-consent-banner"]/div/div/div[2]/a[1]')))
    btn.click()
    # driver.minimize_window()
    # Change filters to show all levels of data
    level = wait.until(
        EC.element_to_be_clickable((By.XPATH, '//select[@id="inputAlert"]'))
    )
    level.send_keys("All")

    # Enter the date to show data after that
    date_field = wait.until(
        EC.element_to_be_clickable((By.XPATH, '//input[@id="inputDateFrom"]'))
    )
    date_field.clear()
    date_field.send_keys(start_date)
    
    # Enter filters
    check_box = wait.until(
            EC.element_to_be_clickable((By.XPATH, f'//input[@id="{filter_tag}"]'))
    )
    check_box.click()
    # time.sleep(10)
    
    # Search Button
    search_button = wait.until(EC.element_to_be_clickable((By.XPATH,'//button[@id="btnsearch"]')))
    actions = ActionChains(driver)
    
    # Double-click the button
    actions.double_click(search_button).perform()
    time.sleep(10)
    
    # Waits until all the event urls are laoded and fetches all event URLs into the list
    urls_data = []
    links = wait.until(
        EC.presence_of_all_elements_located((By.XPATH, '//*[@id="contentResult"]/table/tbody//td[2]/a[@href]'))
    )

    # Loop through the links
    for link in links:
        href = link.get_attribute("href")
        if href:
            # print(href)
            urls_data.append(url_parser(href,filter))
    # Close the WebDriver session
    time.sleep(20)
    # 
    html_downloader(
        # Stores the HTML file in the name format {event_id}_{episode_id}_{event_type}_{event_type_abbreviation} and impact_url
        [[f"{sublist[0]}_{sublist[1]}_{sublist[3]}_{sublist[4]}", sublist[2]] for sublist in urls_data]
        ,filter)
    driver.quit()
    print(filter,filter_tag,"checked and downloaded to local.")


In [6]:
def html_downloader(urls_data,filter):
    # Create a directory to store the HTML files
    # output_dir = f'latest_htmls/{filter}'
    output_dir = html_folder
    absolute_folder_path = Path(output_dir).resolve()
    os.makedirs(absolute_folder_path, exist_ok=True)
    
    for sublist in urls_data:
        response = urllib.request.urlopen(sublist[1])
        webContent = response.read().decode('UTF-8')
        file_path = os.path.join(absolute_folder_path, f"{sublist[0]}.html")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(webContent)
        file.close
        print(f"{sublist[0]}.html")

# Function to point the URL to the  
def url_parser(href,filter):
    parsed_url = urlparse(href)
    # Extract query parameters
    query_params = parse_qs(parsed_url.query)

    # Fetch the eventid and episodeid
    event_id   = query_params.get('eventid',   [None])[0]
    episode_id = query_params.get('episodeid', [None])[0]
    event_type = query_params.get('eventtype', [None])[0]
    # Modify the path of the URL
    new_path = f'/{filter}/report.aspx'
    impact_url = urlunparse((
    parsed_url.scheme,  # https
    parsed_url.netloc,  # www.gdacs.org
    new_path,           # /Filter/report.aspx
    parsed_url.params,  # (empty)
    urlencode(query_params, doseq=True),  # Re-encode the query params
    parsed_url.fragment # (empty)
    ))
    html_info.append([event_id,episode_id ,impact_url,filter])
    # print(impact_url)
    return [event_id,episode_id ,impact_url,filter,event_type]


In [7]:
def html_infofile_csv():
    # Create infoFile
    # Define CSV destination
    destination_folder = csv_folder
    absolute_folder_path = Path(destination_folder).resolve()
    os.makedirs(absolute_folder_path, exist_ok=True)
    info_file_path = f'{absolute_folder_path}\main.csv'
    file_exists = os.path.exists(info_file_path)
    header = ["event_id","episode_id" ,"impact_url","event_type"]
    df = pd.DataFrame(html_info, columns=header)
    # df.head()
    df.to_csv(info_file_path, mode='a' if file_exists else 'w', header=not file_exists, index=False)
    print(f"Html information {'appended to' if file_exists else 'written to'} {info_file_path}")

In [8]:
# Get a list of all files in the specified folder
def list_filenames(folder_path):
    try:
        filenames = os.listdir(folder_path)
        return filenames
    
    except FileNotFoundError:
        print(f"Folder not found: {folder_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Load table configs from file
def load_table_configs(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)


In [9]:
def create_csv(html_file):

    # File and event details
    file_name = html_file.split('/')[-1].split('.')[0]
    event_id, episode, event_type = file_name.split('_')[0:3]
    print(event_id, episode, event_type)

    # Load the table configs
    table_configs_file = f"{event_type}.json"
    table_configs = load_table_configs(table_configs_file)
    
    # Initialize the WebDriver
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(html_file)

    # Process each table
    tables_data = []
    for key, config in table_configs.items():
        xpath = config['xpath']
        col_limit = config['col_limit']
        table_data = extract_table_data(driver, event_id, episode, xpath, col_limit)
        tables_data.append((key, table_data))  # Keep track of table names and data
    
    driver.quit()

    # Define CSV destination
    destination_folder = csv_folder
    absolute_folder_path = Path(destination_folder).resolve()
    os.makedirs(absolute_folder_path, exist_ok=True)

    # Generate CSV files
    for key, table_data in tables_data:
        csv_file_path = f'{absolute_folder_path}\{event_type}_{key}.csv'
        file_exists = os.path.exists(csv_file_path)
        header = table_configs[key]['headers']  # Fetch the headers from the loaded structure
        # print(key, header)
        # print(table_data)
        df = pd.DataFrame(table_data, columns=header)
        # df.head()
        df.to_csv(csv_file_path, mode='a' if file_exists else 'w', header=not file_exists, index=False)
        print(f"Data {'appended to' if file_exists else 'written to'} {csv_file_path}")

In [10]:

# Function to extract table data based on XPath and the number of columns required
def extract_table_data(driver, event_id, episode, xpath, col_limit):
    table_data = []
    if xpath == "//*[@id='alert_summary_left']/table/tbody" or xpath == "//*[@id='tab_responsive']/table/tbody" :
        try:
            temp_tbl=[]
            table = driver.find_element(By.XPATH, xpath)
            rows = table.find_elements(By.XPATH, './/tr')
            if xpath == "//*[@id='alert_summary_left']/table/tbody":
                event_summary = "//*[@id='alert_summary_left']/p"
            else :
                if xpath == "//*[@id='tab_responsive']/table/tbody":
                    event_summary = "//*[@class='p_summary'][1]"
            event_summary_text = driver.find_element(By.XPATH, event_summary).text
            for row in rows:
                columns = row.find_elements(By.XPATH, './/td')
                row_data = [column.text for column in columns]
                if len(row_data)>=1:
                    temp_tbl.append(row_data)
            table_data = [[event_id, episode,*[temp_tbl[i][1] for i in range(1,col_limit)]
                        ,event_summary_text]]
        except NoSuchElementException:
            print(f"Event summary not founds or table with XPath '{xpath}' not found.")
        return table_data
    else:
        try:
            table = driver.find_element(By.XPATH, xpath)
            rows = table.find_elements(By.XPATH, './/tr')
            for row in rows:
                columns = row.find_elements(By.XPATH, './/td' if col_limit else './/th | .//td')
                row_data = []
                for col in columns:
                    cell_text = col.text.strip()

                    # If the cell is empty, look for an image
                    if not cell_text:
                        img_element = col.find_element(By.TAG_NAME, 'img') if col.find_elements(By.TAG_NAME, 'img') else None
                        if img_element:
                            # Try to get the title attribute
                            image_title = img_element.get_attribute('title')
                            if image_title:
                                row_data.append(image_title)  # Add the image title if present
                            else:
                                # Try to get the alt attribute if title is absent
                                alt_text = img_element.get_attribute('alt')
                                if alt_text:
                                    row_data.append(alt_text)  # Add the alt text if present
                                else:
                                    # If no title or alt, extract from the image URL
                                    image_url = img_element.get_attribute('src')
                                    image_code = image_url.split('/')[-1].split('.')[0]  # Extract "8p" from URL
                                    row_data.append(image_code)  # Add the extracted part from the URL
                        else:
                            row_data.append("")  # Leave it blank if no image is founD
                    else:
                        # If the cell contains text, just add it
                        row_data.append(cell_text) 

                if len(row_data) >= 1:
                    table_data.append([event_id, episode] + row_data[:col_limit if col_limit else len(row_data)])
        except NoSuchElementException:
            print(f"Table with XPath '{xpath}' not found.")
    return table_data

In [11]:
def azure_storage_service():
    # Upload Files
    # Create a timestamp subfolder inside the archive files
    datestamp = datetime.now().strftime("%Y-%m-%d-%H:%M:%S")
    az_archive_folder = f"archive/{datestamp}"
    # Archive old CSV files
    blobs_list = container_client.list_blobs(name_starts_with=target_dir_name)
    for blob in blobs_list:
        if blob.name.lower().endswith('.csv'):
            # Get the base file name (remove any subdirectories)
            file_name = os.path.basename(blob.name)
            new_blob_name = f"{az_archive_folder}/{file_name}"
            source_blob_url = f"{acct_url}/{container_name}/{blob.name}?{container_sas_key}"
            container_client.get_blob_client(new_blob_name).start_copy_from_url(source_blob_url)
            # Delete the original blob after copying
            container_client.delete_blob(blob.name)
            # print(f"Moved {blob.name} to {new_blob_name}")
    # Upload latest CSV files
    for root, dirs, files in os.walk(csv_folder):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            blob_client = container_client.get_blob_client(f"{target_dir_name}/{file_name}")
            
            # Upload the file
            with open(file_path, "rb") as data:
                blob_client.upload_blob(data, overwrite=True)
            print(f"Uploaded {file_name} to Azure Blob Storage")

    # Download files from the container folder
    # Delete the directory and its contents and create another one
    if os.path.exists(target_dir_name):
        shutil.rmtree(target_dir_name)
        # print(f'Directory deleted: {target_dir_name}')
    if not os.path.exists(target_dir_name):
        os.makedirs(target_dir_name)
        # print(f'Directory created: {target_dir_name}')

    blobs_list = container_client.list_blobs(name_starts_with=target_dir_name)
    for blob in blobs_list:
        if blob.name.lower().endswith('.csv'):
            # Get the blob client for the current blob
            blob_client = container_client.get_blob_client(blob.name)
            local_dir = os.path.join(os.getcwd(),target_dir_name)
            # Construct the local path to save the file
            local_file_path = os.path.join(local_dir, os.path.relpath(blob.name, target_dir_name))
            # Download the blob to a local file
            with open(local_file_path, "wb") as download_file:
                download_file.write(blob_client.download_blob().readall())
            print(f"Downloaded {blob.name} to {local_file_path}")


In [12]:
def main():
    filter_wise_func_call()
    html_infofile_csv()
    absolute_folder_path = Path(html_folder).resolve()
    filenames = list_filenames(absolute_folder_path)

    for file in filenames:
        absolute_file_path = absolute_folder_path / file
        create_csv(f"file:///{absolute_file_path.as_posix()}")
    
    # azure_storage_service()
    # Shutdown system
    # os.system("shutdown /s /t 1") 
    
if __name__ == "__main__":
    main()

links_iterator Earthquakes inputChEq
Earthquakes inputChEq checking
1450236_1599251_Earthquakes_EQ.html
1450235_1599247_Earthquakes_EQ.html
1450234_1599246_Earthquakes_EQ.html
1450218_1599228_Earthquakes_EQ.html
1450215_1599224_Earthquakes_EQ.html
1450206_1599213_Earthquakes_EQ.html
1450207_1599212_Earthquakes_EQ.html
1450203_1599207_Earthquakes_EQ.html
1450187_1599187_Earthquakes_EQ.html
1450182_1599181_Earthquakes_EQ.html
1450160_1599148_Earthquakes_EQ.html
1450155_1599140_Earthquakes_EQ.html
1450141_1599134_Earthquakes_EQ.html
1450129_1599111_Earthquakes_EQ.html
1450115_1599098_Earthquakes_EQ.html
1450100_1599072_Earthquakes_EQ.html
1450096_1599067_Earthquakes_EQ.html
1450091_1599062_Earthquakes_EQ.html
1450088_1599055_Earthquakes_EQ.html
1450066_1599028_Earthquakes_EQ.html
1450061_1599019_Earthquakes_EQ.html
1450050_1599008_Earthquakes_EQ.html
1450049_1599004_Earthquakes_EQ.html
1450041_1598995_Earthquakes_EQ.html
1450039_1598994_Earthquakes_EQ.html
1450024_1598972_Earthquakes_EQ.h

MaxRetryError: HTTPConnectionPool(host='localhost', port=2763): Max retries exceeded with url: /session/fc3f6c4efb16ff61bb98fa38f036650c/element/f.863073C805B51AAB8F9BB37802C8F66C.d.3193BB6A6CD086D6E40133A861E8EDBB.e.584/text (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000227FACB0910>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))